In [1]:
import requests
from bs4 import BeautifulSoup

In [58]:
def extract_indeed(page_number):
    start = (page_number - 1) * 10
    print('Getting page: ', page_number)
    url = f'https://in.indeed.com/jobs?q=arduino&start={start}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html = response.text
    return BeautifulSoup(html, 'html.parser')


In [54]:
def transform(soup):
    jobs = soup.find_all('div', {'class': 'jobsearch-SerpJobCard'})
    for job in jobs:
        title = job.find('a', {'class': 'jobtitle'})['title']
        company = job.find('span', {'class': 'company'}).text.strip()
        location = job.find('span', {'class': 'location'})
        salary = job.find('span', {'class': 'salaryText'})
        summary = job.find('div', {'class': 'summary'})
        yield dict(
            title = title,
            company = company,
            location = (location or '') and location.text.strip(),
            salary = (salary or '') and salary.text.strip(),
            summary = (summary or '') and summary.text.strip()
        )

In [55]:
import pandas
from itertools import chain

In [56]:
def save_to_file(filename, *, start_page=1, end_page=10):
    joblist = list()
    for page in range(start_page, end_page):
        page_details = extract_indeed(page)
        data = transform(page_details)
        joblist.append(data)
    full_detail = chain.from_iterable(joblist)
    df = pandas.DataFrame(full_detail)
    df.to_csv(filename, index=False)

In [59]:
save_to_file('indeed_arduino_jobs.csv')

Getting page:  1
Getting page:  2
Getting page:  3
Getting page:  4
Getting page:  5
Getting page:  6
Getting page:  7
Getting page:  8
Getting page:  9
